In [1]:
import os
import langextract as lx
from langextract_litellm import LiteLLMLanguageModel
import textwrap
import dotenv
import logging
logging.basicConfig(level=logging.INFO)

dotenv.load_dotenv()
print(os.getenv("AZURE_API_VERSION"))

2024-12-01-preview


In [2]:
prompt = textwrap.dedent("""\
    Extract characters, emotions, and relationships in order of appearance.
    Use exact text for extractions. Do not paraphrase or overlap entities.
    Provide meaningful attributes for each entity to add context.""")

# 2. Provide a high-quality example to guide the model
examples = [
    lx.data.ExampleData(
        text="ROMEO. But soft! What light through yonder window breaks? It is the east, and Juliet is the sun.",
        extractions=[
            lx.data.Extraction(
                extraction_class="character",
                extraction_text="ROMEO",
                attributes={"emotional_state": "wonder"}
            ),
            lx.data.Extraction(
                extraction_class="emotion",
                extraction_text="But soft!",
                attributes={"feeling": "gentle awe"}
            ),
            lx.data.Extraction(
                extraction_class="relationship",
                extraction_text="Juliet is the sun",
                attributes={"type": "metaphor"}
            ),
        ]
    )
]
input_text = "Lady Juliet gazed longingly at the stars, her heart aching for Romeo"

# Azure OpenAI

In [3]:
config = lx.factory.ModelConfig(
    model_id=f"litellm/azure/gpt-4o",
    provider="LiteLLMLanguageModel",
)
model = lx.factory.create_model(config)
results = list(model.infer([prompt]))
results

INFO:absl:Loaded third-party provider class from entry point: litellm
INFO:langextract_litellm.provider:Initialized LiteLLM provider for model: azure/gpt-4o
INFO:langextract_litellm.provider:Calling LiteLLM completion for model azure/gpt-4o
17:38:52 - LiteLLM:INFO: utils.py:3309 - 
LiteLLM completion() model= gpt-4o; provider = azure
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = azure
17:38:53 - LiteLLM:INFO: utils.py:1262 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


[[ScoredOutput(score=1.0, output='Certainly! Please provide the text from which you would like to extract the characters, emotions, and relationships.')]]

In [4]:
# 1. Create model configuration
config = lx.factory.ModelConfig(
    model_id="litellm/azure/gpt-4o",
    provider="LiteLLMLanguageModel",
)
model = lx.factory.create_model(config)

# 2. Use the model parameter (not model_id)
result = lx.extract(
    text_or_documents="Lady Juliet gazed longingly at the stars, her heart aching for Romeo",
    model=model,  # ← Use model parameter, not model_id
    prompt_description=prompt,
    examples=examples
)
print("✅ Extraction successful!")
print(f"Results: {result}")

INFO:langextract_litellm.provider:Initialized LiteLLM provider for model: azure/gpt-4o
DEBUG:absl:Initialized Annotator with prompt:
Extract characters, emotions, and relationships in order of appearance.
Use exact text for extractions. Do not paraphrase or overlap entities.
Provide meaningful attributes for each entity to add context.

Examples
Q: ROMEO. But soft! What light through yonder window breaks? It is the east, and Juliet is the sun.
A: ```json
{
  "extractions": [
    {
      "character": "ROMEO",
      "character_attributes": {
        "emotional_state": "wonder"
      }
    },
    {
      "emotion": "But soft!",
      "emotion_attributes": {
        "feeling": "gentle awe"
      }
    },
    {
      "relationship": "Juliet is the sun",
      "relationship_attributes": {
        "type": "metaphor"
      }
    }
  ]
}
```

Q: 
A: 
INFO:absl:Starting document annotation.
LangExtract: model=azure/gpt-4o [00:00]2025-08-19 17:38:53,393 - langextract.debug - DEBUG - [langextract.

✓ Extraction processing complete



INFO:absl:Finalizing annotation for document ID doc_71bdb774.
INFO:absl:Document annotation completed.


✓ Extracted 3 entities (3 unique types)
  • Time: 1.10s
  • Speed: 62 chars/sec
  • Chunks: 1
✅ Extraction successful!
Results: AnnotatedDocument(extractions=[Extraction(extraction_class='character', extraction_text='Lady Juliet', char_interval=CharInterval(start_pos=0, end_pos=11), alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>, extraction_index=1, group_index=0, description=None, attributes={'emotional_state': 'longing'}), Extraction(extraction_class='emotion', extraction_text='aching', char_interval=CharInterval(start_pos=52, end_pos=58), alignment_status=<AlignmentStatus.MATCH_FUZZY: 'match_fuzzy'>, extraction_index=2, group_index=1, description=None, attributes={'feeling': 'heartache'}), Extraction(extraction_class='relationship', extraction_text='her heart aching for Romeo', char_interval=CharInterval(start_pos=42, end_pos=68), alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>, extraction_index=3, group_index=2, description=None, attributes={'type': 'ro